# Imputación básica

## Preparar datos

In [60]:
import pandas as pd
import numpy as n
import matplotlib.pyplot as plt
import random
import os
import warnings
import math

## Leer datos originales

Hay que leer el dataset original para comparar los datos imputados con los reales

Temperatura

In [61]:
warnings.filterwarnings('ignore', category=SyntaxWarning)  # Porque el \d de \data y \daily lo toma como un valor especial
dataframe_temperatura = pd.read_csv("..\data\daily-minimum-temperatures-in-me.csv", header=0, index_col=0, parse_dates=True)
series_temperatura = dataframe_temperatura.squeeze('columns')
series_temperatura = series_temperatura.resample('D').mean()
series_temperatura['1984-12-31'] = (series_temperatura['1984-12-30'] + series_temperatura['1985-01-01'])/2
series_temperatura['1988-12-31'] = (series_temperatura['1988-12-30'] + series_temperatura['1989-01-01'])/2
series_temperatura.head()

Date
1981-01-01    20.7
1981-01-02    17.9
1981-01-03    18.8
1981-01-04    14.6
1981-01-05    15.8
Freq: D, Name: Daily minimum temperatures in Melbourne, Australia, 1981-1990, dtype: float64

Cardiovascular

In [62]:
dataframe_cardiovascular = pd.read_csv("..\data\\ads_hour.csv", header=0, index_col=0, parse_dates=True)
series_cardiovascular = dataframe_cardiovascular.squeeze('columns')
series_cardiovascular.head()

C:\Users\alaba\AppData\Local\Temp\ipykernel_7104\1350282516.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataframe_cardiovascular = pd.read_csv("..\data\\ads_hour.csv", header=0, index_col=0, parse_dates=True)


Date
2017-08-03 00:00:00    49136
2017-08-03 01:00:00    46450
2017-08-03 02:00:00    46355
2017-08-03 03:00:00    43748
2017-08-03 04:00:00    42281
Name: ads, dtype: int64

Manchas solares

In [63]:
dataframe_sunspots = pd.read_csv("..\data\monthly-sunspots.csv", header=0, index_col=0, parse_dates=True)
series_sunspots = dataframe_sunspots.squeeze('columns')
series_sunspots = series_sunspots.resample('ME').mean() # Para los meses
series_sunspots.head()

Month
1749-01-31    58.0
1749-02-28    62.6
1749-03-31    70.0
1749-04-30    55.7
1749-05-31    85.0
Freq: ME, Name: Sunspots, dtype: float64

## Funciones auxiliares

### Listar dataset

In [64]:
def listar_datasets(ruta):
    return os.listdir(ruta)

### Calcular RMSE

In [65]:
def RMSE(serie_original, serie_nula, serie_imputada):
    suma=0
    for i in serie_nula[serie_nula.isna()].index:
        suma += (serie_original[i] - serie_imputada[i])**2
    suma = math.sqrt(suma/len(serie_original))
    return suma

### Calcular MAE

In [66]:
def MAE(serie_original, serie_nula, serie_imputada):
    suma=0
    for i in serie_nula[serie_nula.isna()].index:
        suma += abs(serie_original[i] - serie_imputada[i])
    suma = suma/len(serie_original)
    return suma

### Imputar serie

In [ ]:
def imputar_media_mediana(serie, ruta, lista):
    for i in lista:
        # Lo leemos
        df = pd.read_csv(ruta+i, header=0, index_col=0, parse_dates=True)
        serie_nulo = df.squeeze('columns')
        
        # Imputamos con la media:
        serie_media = serie_nulo.fillna(serie_nulo.mean())
        # Imputamos con la mediana
        serie_mediana = serie_nulo.fillna(serie_nulo.median())

        # Calculamos la raiz del error cuadrático medio con las series imputadas con la media y la mediana
        rmse_media = RMSE(serie, serie_nulo, serie_media)
        rmse_mediana = RMSE(serie, serie_nulo, serie_mediana)
        # Calculamos el error medio absoluto con las series imputadas con la media y la mediana
        mae_media = MAE(serie, serie_nulo, serie_media)
        mae_mediana = MAE(serie, serie_nulo, serie_mediana)
        print("El RMSE imputando con la media el dataset " + i + " es : " + str(rmse_media))
        print("El RMSE imputando con la mediana el dataset " + i + " es : " + str(rmse_mediana))
        print("El MAE imputando con la media el dataset " + i + " es : " + str(mae_media))
        print("El MAE imputando con la mediana el dataset " + i + " es : " + str(mae_mediana))
        print("------------------------------------------------------------------------------------------------------------------")

## Imputamos

### Dataset de temperaturas

In [68]:
lista_datasets_temperatura = listar_datasets('..\miss_valued_data_temperatura')
imputar_media_mediana(series_temperatura, "..\miss_valued_data_temperatura\\", lista_datasets_temperatura)

min_temperature_bateria_10.csv
El RMSE imputando con la media el dataset min_temperature_bateria_10.csv es : 1.4747579259701238
El RMSE imputando con la mediana el dataset min_temperature_bateria_10.csv es : 1.4694751449185541
El MAE imputando con la media el dataset min_temperature_bateria_10.csv es : 0.37715499689939913
El MAE imputando con la mediana el dataset min_temperature_bateria_10.csv es : 0.37487677984665946
------------------------------------------------------------------------------------------------------------------
min_temperature_bateria_15.csv
El RMSE imputando con la media el dataset min_temperature_bateria_15.csv es : 1.6719544860161197
El RMSE imputando con la mediana el dataset min_temperature_bateria_15.csv es : 1.668904325721468
El MAE imputando con la media el dataset min_temperature_bateria_15.csv es : 0.5193103772137296
El MAE imputando con la mediana el dataset min_temperature_bateria_15.csv es : 0.5173466593647317
------------------------------------------

### Dataset cardiovascular

In [69]:
lista_datasets_cardiovascular = listar_datasets('..\miss_valued_data_cardiovascular')
imputar_media_mediana(series_cardiovascular, "..\miss_valued_data_cardiovascular\\", lista_datasets_cardiovascular)

cardiovascular_arritmias10.csv
El RMSE imputando con la media el dataset cardiovascular_arritmias10.csv es : 5492.510722300351
El RMSE imputando con la mediana el dataset cardiovascular_arritmias10.csv es : 5404.674501542706
El MAE imputando con la media el dataset cardiovascular_arritmias10.csv es : 1406.950486834829
El MAE imputando con la mediana el dataset cardiovascular_arritmias10.csv es : 1376.3640287769783
------------------------------------------------------------------------------------------------------------------
cardiovascular_arritmias15.csv
El RMSE imputando con la media el dataset cardiovascular_arritmias15.csv es : 6959.605598683248
El RMSE imputando con la mediana el dataset cardiovascular_arritmias15.csv es : 6899.6281406070775
El MAE imputando con la media el dataset cardiovascular_arritmias15.csv es : 2162.598832744293
El MAE imputando con la mediana el dataset cardiovascular_arritmias15.csv es : 2128.2081534772183
------------------------------------------------

### Dataset manchas solares

In [70]:
lista_datasets_sunspots = listar_datasets('..\miss_valued_data_manchas_solares')
imputar_media_mediana(series_sunspots, "..\miss_valued_data_manchas_solares\\", lista_datasets_sunspots)

manchas_solares_error_medida_10.csv
El RMSE imputando con la media el dataset manchas_solares_error_medida_10.csv es : 14.474973789939577
El RMSE imputando con la mediana el dataset manchas_solares_error_medida_10.csv es : 14.95440208914992
El MAE imputando con la media el dataset manchas_solares_error_medida_10.csv es : 3.5866186587976228
El MAE imputando con la mediana el dataset manchas_solares_error_medida_10.csv es : 3.5486524822695036
------------------------------------------------------------------------------------------------------------------
manchas_solares_error_medida_15.csv
El RMSE imputando con la media el dataset manchas_solares_error_medida_15.csv es : 18.082540006822114
El RMSE imputando con la mediana el dataset manchas_solares_error_medida_15.csv es : 18.593063127249927
El MAE imputando con la media el dataset manchas_solares_error_medida_15.csv es : 5.4840606905203595
El MAE imputando con la mediana el dataset manchas_solares_error_medida_15.csv es : 5.39429078014